# `polars`:  Write query for larger than memory data sets

[`polars`](https://www.pola.rs) loads in *lazy mode* only the necessary data into memory. This is done via `scan` instead of `read`, which creates a lazy `polars` query. By calling `collect` in the end the selected data is loaded into memory. When setting `collect(streaming=True)` the data is collected in batches (not available for all lazy operations). This can be used for larger than memory datasets. Eager DataFrames can be converted with `.lazy()`.

Eager mode:
```
# Full dataset is loaded to memory
df = pl.read_csv(„docs/data/iris.csv")

df_small = df.filter(pl.col("sepal_length") > 5)
df_agg = (
    df_small.group_by(„species“)
    .agg(pl.col(„sepal_width")
    .mean()
)
```

Lazy mode:
```
# Define lazy data frame (no computation will happen here)
df = pl.scan_csv("iris.csv")
df = df.filter(pl.col("sepal length (cm)") > 5)
df = df.group_by("target").agg(pl.col("sepal width (cm)").mean())

# Data is collected (now the computation happens)
df = df.collect(streaming=True)
```

**When not to use lazy mode?** During exploration, when intermediate results are needed and the design of the query is not completely known yet.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import random
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import polars as pl

## Generate Data
This simulated dataset contains a timeseries of electrical voltage and current values for different substations.

In [ ]:
# Fix the random seeds to have the same outcome when the cell is executed again
np.random.seed(42)
random.seed(42)

# Number of generated data points
n = 1_000_000

# Generate timestamps by subtraction from the current time the desired number of generated data points in minutes
timestamps = [datetime.now() - timedelta(minutes=i) for i in range(n)]
# Generate substation ids in the range of 1 to 10
substation_ids = [random.randint(1, 10) for _ in range(n)]
# Generate voltages as normal distribution with mean 230 and standard deviation 10
voltages = np.random.normal(230, 10, n)
# Generate currents as normal distribution with mean 5 and standard deviation 2
currents = np.random.normal(5, 2, n)

# Collect all lists and create a DataFrame
df = pd.DataFrame(
    {
        "timestamp": timestamps,
        "substation_id": substation_ids,
        "voltage": voltages,
        "current": currents,
    }
)

# Write the DataFrame to a CSV file
df.to_csv("electricity_usage.csv", index=False)

## Exercise: "How can we make the code even faster?" - polars version with lazy mode
 
Now, we imagine that there is a larger than memory data set. Think about updating the number of generated data points above to e.g. 4_000_000 (not more than that, please).
Write the previous `polars` code as a `polars` query/pipeline (either use your code or the provided solution below as base).

Execute the query in lazy mode and measure the run time with `%%timeit`


In [ ]:
%%timeit
# Step 1: Read data from csv
df = pl.read_csv("electricity_usage.csv")

# Step 2: Set the correct timestamp format
df = df.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S%.f"))

# Step 3: Calculate power (P=VI) and add it as a new column
df = df.with_columns((pl.col("voltage") * pl.col("current")).alias("power"))

# Step 4: Group by 'substation_id', resample timestamp and calculate daily average power
df = df.sort("timestamp")  # data has to be sorted for group_by_dynamic!
df_grouped = df.group_by_dynamic(
    index_column="timestamp",
    every="1d",
    closed="right",
    by="substation_id",
    include_boundaries=False,
).agg(pl.col("power").mean().alias("daily_avg_power"))

# Step 5: Filter out data where daily average power is less than a certain threshold
threshold = 1000
df_grouped = df_grouped.filter(pl.col("daily_avg_power") > threshold)

# Step 6: Sort result by substation and timestamp
df_grouped = df_grouped.sort(["substation_id", "timestamp"])

# Step 7: Write the transformed data to a new CSV file
df_grouped.write_csv("transformed_electricity_usage.csv")

In [ ]:
# Your code goes here!

---
_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © [Point 8 GmbH](https://point-8.de)_